# Tutorial: StructureMap

## Import libraries

In [1]:
# Import structuremap functions
import structuremap.utils
structuremap.utils.set_logger()
from structuremap.processing import download_alphafold_cif, download_alphafold_pae, format_alphafold_data, annotate_accessibility, get_smooth_score, annotate_proteins_with_idr_pattern, get_extended_flexible_pattern, get_proximity_pvals, perform_enrichment_analysis, perform_enrichment_analysis_per_protein

In [2]:
# Import 
import pandas as pd
import numpy as np
import os
import re
import plotly.express as px
import tqdm
import tempfile

In [3]:
# Set output directory to tempdir
output_dir = tempfile.gettempdir()

## Select proteins to use for this tutorial

In [4]:
test_proteins = ['O43353','P24941','Q92918','P45984','P28482','O96017',
                 'P02730','Q8NB16','Q13546','P29320','P08559','P15121']

## Download AlphaFold data

In [5]:
cif_dir = os.path.join(output_dir, 'tutorial_cif')
pae_dir = os.path.join(output_dir, 'tutorial_pae')

In [6]:
valid_proteins_cif, invalid_proteins_cif, existing_proteins_cif = download_alphafold_cif(
    proteins=test_proteins,
    out_folder=cif_dir)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 7962.61it/s]

2022-02-07 08:52:41> Valid proteins: 0
2022-02-07 08:52:41> Invalid proteins: 0
2022-02-07 08:52:41> Existing proteins: 12


In [7]:
valid_proteins_pae, invalid_proteins_pae, existing_proteins_pae = download_alphafold_pae(
    proteins=test_proteins,
    out_folder=pae_dir, 
    )

100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 16666.11it/s]

2022-02-07 08:52:41> Valid proteins: 0
2022-02-07 08:52:41> Invalid proteins: 0
2022-02-07 08:52:41> Existing proteins: 12


## Format AlphaFold data input

In [8]:
alphafold_annotation = format_alphafold_data(
    directory=cif_dir, 
    protein_ids=test_proteins)

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  4.66it/s]


In [9]:
alphafold_annotation[0:3]

,protein_id,protein_number,AA,position,quality,x_coord_c,x_coord_ca,x_coord_cb,x_coord_n,y_coord_c,...,z_coord_ca,z_coord_cb,z_coord_n,secondary_structure,structure_group,BEND,HELX,STRN,TURN,unstructured
0,O43353,1,M,1,36.95,-20.649,-22.109,-22.991,-22.332,-11.261,...,-31.320,-30.655,-30.726,unstructured,unstructured,0,0,0,0,1
1,O43353,1,N,2,39.38,-18.674,-18.530,-17.736,-19.886,-8.995,...,-32.003,-33.314,-32.160,unstructured,unstructured,0,0,0,0,1
2,O43353,1,G,3,46.56,-17.165,-18.521,NaN,-18.591,-6.766,...,-29.807,NaN,-30.291,unstructured,unstructured,0,0,0,0,1


## Annotate pPSE values

In [10]:
full_sphere_exposure = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=24, 
    max_angle=180, 
    error_dir=pae_dir)

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:03<00:00,  3.88it/s]


In [11]:
alphafold_accessibility = alphafold_annotation.merge(
    full_sphere_exposure, how='left', on=['protein_id','AA','position'])

In [12]:
part_sphere_exposure = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=12, 
    max_angle=70, 
    error_dir=pae_dir)

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 65.21it/s]


In [13]:
alphafold_accessibility = alphafold_accessibility.merge(
    part_sphere_exposure, how='left', on=['protein_id','AA','position'])

In [14]:
alphafold_accessibility['high_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae <= 5, 1, 0)
alphafold_accessibility['low_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae > 5, 1, 0)

## Annotate IDRs

In [15]:
alphafold_accessibility_smooth = get_smooth_score(
    alphafold_accessibility, 
    np.array(['nAA_24_180_pae']), 
    [10])

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 39.58it/s]


In [16]:
alphafold_accessibility_smooth['IDR'] = np.where(
    alphafold_accessibility_smooth['nAA_24_180_pae_smooth10']<=34.27, 1, 0)

## Annottate short IDRs

In [17]:
alphafold_accessibility_smooth_pattern = annotate_proteins_with_idr_pattern(
    alphafold_accessibility_smooth,
    min_structured_length = 80, 
    max_unstructured_length = 20)

100%|███████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 567.99it/s]


In [18]:
alphafold_accessibility_smooth_pattern_ext = get_extended_flexible_pattern(
    alphafold_accessibility_smooth_pattern, 
    ['flexible_pattern'], [5])

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 29.54it/s]


In [19]:
alphafold_accessibility_smooth_pattern_ext[0:3]

,protein_id,protein_number,AA,position,quality,x_coord_c,x_coord_ca,x_coord_cb,x_coord_n,y_coord_c,...,TURN,unstructured,nAA_24_180_pae,nAA_12_70_pae,high_acc_5,low_acc_5,nAA_24_180_pae_smooth10,IDR,flexible_pattern,flexible_pattern_extended_5
0,O43353,1,M,1,36.95,-20.649,-22.109,-22.991,-22.332,-11.261,...,0,1,4,0,1,0,29.909091,1,0,0
1,O43353,1,N,2,39.38,-18.674,-18.530,-17.736,-19.886,-8.995,...,0,1,5,0,1,0,34.500000,0,0,0
2,O43353,1,G,3,46.56,-17.165,-18.521,NaN,-18.591,-6.766,...,0,1,6,0,1,0,38.461538,0,0,0


## Annotate PTM data

In [21]:
ptm_file_location = os.path.join("..","data","test_files","ptm_file.csv")
ptm_file = pd.read_csv(ptm_file_location)

In [22]:
ptm_file[0:3]

,Unnamed: 0,protein_id,AA,position,ac,ac_reg,ga,gl,gl_reg,m,m_reg,p,p_reg,sm,sm_reg,ub,ub_reg
0,0,O43353,K,17,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,O43353,K,182,0,0,0,0,0,0,0,0,0,0,0,1,0
2,2,O43353,K,203,0,0,0,0,0,0,0,0,0,0,0,1,0


In [23]:
alphafold_ptms = alphafold_accessibility_smooth_pattern_ext.merge(ptm_file, how='left', on=['protein_id','AA','position'])
alphafold_ptms = alphafold_ptms.fillna(0)

In [24]:
ptm_site_dict = {'p':['S','T','Y'],
                 'p_reg':['S','T','Y']}

## Perform PTM enrichment analysis

In [25]:
enrichment_p = perform_enrichment_analysis(
    df=alphafold_ptms, 
    ptm_types=['p', 'p_reg'], 
    rois=['IDR'], 
    ptm_site_dict = ptm_site_dict,
    quality_cutoffs=[0])

In [26]:
enrichment_p

,quality_cutoff,ptm,roi,n_aa_ptm,n_aa_roi,n_ptm_in_roi,n_ptm_not_in_roi,n_naked_in_roi,n_naked_not_in_roi,oddsr,p,p_adj_bf,p_adj_bh
0,0,p,IDR,281,259,116,165,143,639,3.141513,1.209934e-13,2.419868e-13,2.419868e-13
0,0,p_reg,IDR,60,259,21,39,238,765,1.730769,6.193364e-02,1.238673e-01,6.193364e-02


## Perform PTM enrichment analysis per protein

In [27]:
enrichment_p_per_protein = perform_enrichment_analysis_per_protein( 
    df=alphafold_ptms, 
    ptm_types=['p', 'p_reg'], 
    rois=['IDR'], 
    ptm_site_dict = ptm_site_dict,
    quality_cutoffs=[0])

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 95.06it/s]


In [28]:
enrichment_p_per_protein

,protein_id,quality_cutoff,ptm,roi,n_aa_ptm,n_aa_roi,n_ptm_in_roi,n_ptm_not_in_roi,n_naked_in_roi,n_naked_not_in_roi,oddsr,p,p_adj_bf,p_adj_bh
0,O43353,0,p,IDR,23,32,19,4,13,57,20.826923,5.076834e-08,6.092200e-07,3.046100e-07
1,O43353,0,p_reg,IDR,2,32,1,1,31,60,1.935484,1.000000e+00,1.000000e+00,1.000000e+00
2,O96017,0,p_reg,IDR,21,40,9,12,31,51,1.233871,8.025566e-01,1.000000e+00,1.000000e+00
3,P02730,0,p_reg,IDR,4,17,4,0,13,107,inf,2.536975e-04,3.044370e-03,1.014790e-03
4,P29320,0,p,IDR,23,37,8,15,29,128,2.354023,9.479941e-02,1.000000e+00,2.843982e-01
5,P29320,0,p_reg,IDR,5,37,2,3,35,140,2.666667,2.730826e-01,1.000000e+00,6.553982e-01
6,P45984,0,p,IDR,11,21,3,8,18,42,0.875000,1.000000e+00,1.000000e+00,1.000000e+00
7,Q13546,0,p,IDR,36,62,21,15,41,37,1.263415,6.863530e-01,1.000000e+00,1.000000e+00
8,Q13546,0,p_reg,IDR,4,62,3,1,59,51,2.593220,6.242301e-01,1.000000e+00,1.000000e+00
9,Q8NB16,0,p_reg,IDR,3,4,0,3,4,57,0.000000,1.000000e+00,1.000000e+00,1.000000e+00


## Get 3D clusters

In [29]:
proximity_res_p = get_proximity_pvals(
    df=alphafold_ptms, 
    ptm_types = ['p'], 
    ptm_site_dict = ptm_site_dict, 
    error_dir=pae_dir, 
    per_site_metric= 'mean',
    error_operation='plus',
    n_random=10000, 
    random_seed=44)

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:13<00:00,  1.10s/it]


In [30]:
proximity_res_p

,protein_id,ptm,n_ptms,pvalue_1d,pvalue_3d,pvalue_1d_adj_bh,pvalue_3d_adj_bh
0,O43353,p,23.0,0.1935,0.9943,0.290250,1.000000
1,O96017,p,42.0,0.1384,0.8308,0.276800,1.000000
2,P02730,p,24.0,0.8482,0.4442,0.848200,0.761486
3,P08559,p,21.0,0.0000,0.0003,0.000000,0.003600
4,P15121,p,16.0,0.3715,0.0016,0.445800,0.009600
5,P24941,p,11.0,0.0863,0.0523,0.207120,0.192720
6,P28482,p,21.0,0.2846,0.8203,0.379467,1.000000
7,P29320,p,23.0,0.0004,0.0803,0.001600,0.192720
8,P45984,p,11.0,0.0367,0.2277,0.110100,0.455400
9,Q13546,p,36.0,0.1683,0.0765,0.288514,0.192720
